## **Modules**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas import DataFrame

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

## **Data**

**Import the normalized data**

In [2]:
file_path = "normalized_data.csv"
df = pd.read_csv(file_path)
df

,Unnamed: 0,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,...,low_MeanDev48_over_StdDev48,close_MeanDev48_over_StdDev48,open_MeanDev96_over_StdDev96,high_MeanDev96_over_StdDev96,low_MeanDev96_over_StdDev96,close_MeanDev96_over_StdDev96,open_MeanDev192_over_StdDev192,high_MeanDev192_over_StdDev192,low_MeanDev192_over_StdDev192,close_MeanDev192_over_StdDev192
0,359,0.018495,0.009785,0.018383,0.004281,1180.825933,-0.241505,11017.0,612.021981,0.000000,...,0.370477,0.661152,-1.950690,-1.881119,-1.871933,-1.950690,0.593925,0.969559,-0.596205,0.593925
1,360,0.004956,-0.008991,-0.004450,-0.003945,693.787209,-0.732426,8059.0,343.998248,0.000000,...,0.480687,0.630321,-1.811761,-1.745879,-1.770050,-1.811761,0.371924,0.791939,-1.087814,0.371924
2,361,-0.003946,0.004507,-0.000473,-0.003816,639.808801,-0.784803,6949.0,297.590606,0.000000,...,0.389880,0.548217,-1.678097,-1.709976,-1.709619,-1.678097,-0.010608,0.631068,-1.541996,-0.010608
3,362,-0.003369,-0.005212,0.002062,0.006863,463.446578,-0.958557,5328.0,232.254343,0.000000,...,0.350033,0.460780,-1.660865,-1.701925,-1.692418,-1.660865,-0.052136,0.782304,-1.830356,-0.052136
4,363,0.006419,0.004713,0.002860,-0.002752,634.129456,-0.787283,7406.0,318.341325,0.000000,...,0.247121,0.354412,-1.587490,-1.630426,-1.655253,-1.587490,-0.142859,0.554341,-1.989684,-0.142859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38443,38802,-0.003626,-0.002886,-0.009204,-0.008336,2613.774410,-0.703874,42193.0,1245.791750,-0.887751,...,0.894451,1.038517,-0.844173,-0.114303,-1.152432,-0.844173,-1.239710,-1.730291,-2.099255,-1.239710
38444,38803,-0.008337,-0.009908,-0.006327,-0.006609,4195.854000,-0.346556,58827.0,1911.811000,-0.560005,...,0.435777,0.910978,-0.640086,-0.109049,-1.067451,-0.640086,-2.261169,-2.252223,-2.587937,-2.261169
38445,38804,-0.006609,-0.000843,-0.001635,-0.008575,3271.915890,-0.552267,53095.0,1526.045120,-0.747297,...,0.309717,0.506155,-0.569034,0.065389,-0.977107,-0.569034,-2.692674,-2.448695,-2.436019,-2.692674
38446,38805,-0.008575,-0.009336,-0.002849,0.002702,3588.324010,-0.482464,53154.0,1650.636910,-0.678239,...,0.400968,0.594548,-0.490384,0.124920,-0.895412,-0.490384,-2.263128,-2.371402,-2.230504,-2.263128


**Define the periods of time that the features have in the data**

For example, the features contain SMA2 which is the smoothed moving average during 2 days, so a period of time that must be in the periods list is 2.

In [3]:
periods = [2,4,8,12,24,48,96,192]

**Label value counts**

In [4]:
df["label"].value_counts()

True     19580
False    18868
Name: label, dtype: int64

### **Data Preprocessing**

**Drop useless column**

In [5]:
df.drop(columns=["Unnamed: 0"], inplace=True)

**Drop nan if exist**

In [6]:
# Print all the amount of nan values
df.isna().sum().sum()

0

Or forward fill before drop, this forward filling can avoid data leakeage!

In [7]:
forward_fill = False

if forward_fill:
    df.ffill(inplace=True)
else:
    df.dropna(inplace=True)

**Create train set, validation set and test set**

In [8]:
X_df = df.drop(columns=["label"])
y_df = df["label"]

In [9]:
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=0.3, shuffle=False)
X_val_df, X_test_df, y_val_df, y_test_df = train_test_split(X_test_df, y_test_df, test_size=0.5, shuffle=False)

## **Model and training**

### Random Forest

**Training**

In [10]:
model_rf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, 
                                  min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', bootstrap=True, 
                                  oob_score=False, n_jobs=-1, random_state=1234, verbose=1)

In [11]:
model_rf.fit(X_train_df, y_train_df)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    8.8s finished


RandomForestClassifier(max_features='sqrt', n_jobs=-1, random_state=1234,
                       verbose=1)

**Evaluation**

In [12]:
print("AUC score in the train set:", roc_auc_score(y_train_df, model_rf.predict_proba(X_train_df)[:, 1]))

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.1s


AUC score in the train set: 1.0


[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.2s finished


In [13]:
print("AUC score in the validation set:", roc_auc_score(y_val_df, model_rf.predict_proba(X_val_df)[:, 1]))

AUC score in the validation set: 0.5383297974769913


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished


In [14]:
print("AUC score in the test set:", roc_auc_score(y_test_df, model_rf.predict_proba(X_test_df)[:, 1]))

AUC score in the test set: 0.5220341793638587


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished


In [15]:
print("Accuracy score in the train set:", accuracy_score(y_train_df, (model_rf.predict_proba(X_train_df)[:, 1] >= 0.5) * 1))

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.1s


Accuracy score in the train set: 1.0


[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.2s finished


In [16]:
print("Accuracy score in the validation set:", accuracy_score(y_val_df, (model_rf.predict_proba(X_val_df)[:, 1] >= 0.5) * 1))

Accuracy score in the validation set: 0.5280041616091555


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished


In [17]:
print("Accuracy score in the test set:", accuracy_score(y_test_df, (model_rf.predict_proba(X_test_df)[:, 1] >= 0.5) * 1))

Accuracy score in the test set: 0.5090152565880721


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished


**Threshold tuning**

In [18]:
pred_val = model_rf.predict_proba(X_val_df)[:, 1]
tmp_pred_val = pred_val.copy()

l = []
for i in range(4000, 6000):
    pred_val = (tmp_pred_val >= i/10000) * 1
    l.append(accuracy_score(y_val_df, pred_val))

threshold = max(l)
print(max(l))

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished


0.5280041616091555


In [19]:
pred_test = model_rf.predict_proba(X_test_df)[:,1]
pred_test = (pred_test >= threshold) * 1
print("Final testing accuracy:")
accuracy_score(y_test_df, pred_test)

Final testing accuracy:


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished


0.5176837725381415

**Features importance**

In [20]:
importance_df = pd.DataFrame()
importance_df['feature'] = model_rf.feature_names_in_
importance_df['importance'] = model_rf.feature_importances_
importance_df.sort_values('importance', ascending=False).head(20)

,feature,importance
32,BOP,0.002056
71,close_EMA2,0.001961
144,close_ALMA4,0.001760
68,close_WMA2,0.001751
66,close_SMA2,0.001679
42,ForceIndex2,0.001456
33,UO,0.001452
72,close_DEMA2,0.001449
1438,log_volume_resid_divide_prev,0.001427
67,close_SMMA2,0.001422


**Note:**

The result is different at different training times, in the reality, we train multiple times the model and choose the one that gives us the best accuracy on the validation set